<img src="Bilder/ost_logo.png" width="240" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN05/5.5-Channels.ipynb)

# Multiple Input and Multiple Output Channels


- Farbbilder haben die Standard-RGB-Kanäle, um den Anteil von Rot, Grün und Blau anzugeben.
- Bis jetzt haben wir alle unsere numerischen Beispiele vereinfacht, indem wir nur mit einem einzigen Eingabe- und einem einzigen Ausgabekanal gearbeitet haben.
- Dies hat es uns ermöglicht, unsere Eingänge, Faltungskerne, und Ausgänge jeweils als zweidimensionale Tensoren zu betrachten.

Wenn wir Kanäle in den Mix einbeziehen, werden unsere Eingaben und versteckten Repräsentationen
zu dreidimensionalen Tensoren. 

Zum Beispiel hat jedes **RGB-Eingabebild die Form $3\times h\times w$.** Wir bezeichnen diese Achse mit einer Grösse von 3 als die *Kanaldimension* (*channel dimension*).

## Mehrere Eingangskanäle

- Wenn die Eingabedaten mehrere Kanäle enthalten, müssen wir einen Faltungs-Kernel mit der gleichen Anzahl von Eingangskanälen wie die Eingabedaten konstruieren, damit er eine Kreuzkorrelation mit den Eingabedaten durchführen kann.
- Angenommen, die Anzahl der Kanäle für die Eingabedaten ist $c_i$, dann muss die Anzahl der Eingangskanäle des Faltungskerns ebenfalls $c_i$ betragen.
- Wenn die Fensterform unseres Faltungskerns $k_h\times k_w$ ist, dann können wir, wenn $c_i=1$ ist, unseren Faltungs-Kernel als einen zweidimensionalen Tensor der Form $k_h\times k_w$ betrachten.



Wenn jedoch $c_i>1$, brauchen wir einen Kernel der für *jeden* Eingangskanal einen Tensor der Form $k_h\times k_w$ enthält. Die Verkettung dieser $c_i$-Tensoren zusammen ergibt einen Faltungs-Kernel der Form $c_i\times k_h\times k_w$.

**Da der Eingang und der Faltungskern jeweils $c_i$ Kanäle haben, können wir für jeden Kanal eine Kreuzkorrelation
auf dem 2D Tensor der Eingabe und dem 2D Tensor des Faltungskerns durchführen, wobei die $c_i$ Ergebnisse addiert werden (Summierung über die Kanäle), um einen 2D Tensor zu erhalten.**

Dies ist das Ergebnis einer zweidimensionalen Kreuzkorrelation zwischen einem Mehrkanaleingang und
einem Faltungs-Kernel mit mehreren Eingangskanälen.



In der nächsten Abbildung zeigen wir ein Beispiel
für eine zweidimensionale Kreuzkorrelation mit zwei Eingangskanälen.
Die schattierten Bereiche sind das erste Ausgabeelement sowie die Eingabe- und Kernel-Tensorelemente, die für die Ausgabeberechnung verwendet werden: $(1\cdot 1+2\cdot2+4\cdot3+5\cdot4)+(0\cdot0+1\cdot1+3\cdot2+4\cdot3)=56$.

<img src="Bilder/conv-multi-in.svg" width="440" height="340" align="center"/>

Um sicherzustellen, dass wir wirklich verstehen, was hier vor sich geht,
können wir **Kreuzkorrelationsoperationen mit mehreren Eingangskanälen** selbst durchführen.
Beachten Sie, dass wir lediglich eine Kreuzkorrelationsoperation pro Kanal durchführen und dann die Ergebnisse addieren.

In [ ]:
import torch

def corr2d(X, K):
    """Compute 2D cross-correlation (convolution without flipping)."""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))  # Output size

    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = torch.sum(X[i:i + h, j:j + w] * K)

    return Y

def corr2d_multi_in(X, K):
    """
    Compute cross-correlation with multiple input channels.

    Parameters:
    - X (torch.Tensor): Multi-channel input tensor (C_in, H, W)
    - K (torch.Tensor): Multi-channel kernel (C_in, kH, kW)

    Returns:
    - torch.Tensor: Output after summing over all channels.
    """
    # Compute the sum of 2D correlations over all input channels
    return torch.stack([corr2d(x, k) for x, k in zip(X, K)], dim=0).sum(dim=0)


Wir können den Eingabe-Tensor `X` und den Kernel-Tensor `K` konstruieren
entsprechend den Werten in obiger Figur zu konstruieren, um **die Ausgabe** der Kreuzkorrelationsoperation zu validieren.


In [ ]:
# Define multi-channel input tensor X (2 input channels, 3x3)
X = torch.tensor([[[0.0, 1.0, 2.0], 
                   [3.0, 4.0, 5.0], 
                   [6.0, 7.0, 8.0]],

                  [[1.0, 2.0, 3.0], 
                   [4.0, 5.0, 6.0], 
                   [7.0, 8.0, 9.0]]])  # Shape: (2, 3, 3)

# Define kernel tensor K (2 input channels, 2x2 kernel)
K = torch.tensor([[[0.0, 1.0], 
                   [2.0, 3.0]],

                  [[1.0, 2.0], 
                   [3.0, 4.0]]])  # Shape: (2, 2, 2)

# Perform cross-correlation
output = corr2d_multi_in(X, K)

# Print the result
print("Output:\n", output)


## Mehrere Ausgangskanäle

- Unabhängig von der Anzahl der Eingangskanäle, hatten wir immer bisher nur einen Ausgangskanal.
- In den gängigsten Architekturen neuronaler Netze, wird die Kanaldimension je höher wir im neuronalen Netz aufsteigen in der Regel erhöht. Man gewinnt **Kanaltiefe, und reduziert gleichzeitig die räumliche Auflösung durch Donwsampling**, je weiter wir im Netz aufsteigen.
- Intuitiv könnte man sich vorstellen, dass jeder Kanal auf eine andere Gruppe von Merkmalen reagiert.
- Die Realität ist etwas komplizierter, da die Repräsentationen nicht unabhängig voneinander gelernt werden, sondern vielmehr so optimiert werden, dass sie gemeinsam nützlich sind.
- Es kann also sein, dass nicht ein einzelner Kanal einen Kantendetektor erlernt, sondern dass eine bestimmte Richtung im Kanalraum der Erkennung von Kanten entspricht.



Bezeichnen Sie mit $c_i$ und $c_o$ die Anzahl der Eingabe- bzw. Ausgabekanäle, und lassen Sie $k_h$ und $k_w$ die Höhe und Breite des Kernels sein.
- Um eine Ausgabe mit mehreren Kanälen zu erhalten, können wir einen Kernel-Tensor der Form $c_i\times k_h\times k_w$ für *jeden* Ausgabekanal erstellen.
- Wir konkatenieren sie auf der Dimension des Ausgangskanals, so dass die Form des Faltungskerns
$c_o\times c_i\times k_h\times k_w$ ist.

**Bei Kreuzkorrelationsoperationen, wird das Ergebnis für jeden Ausgangskanal aus dem Faltungs-Kernel berechnet, der diesem Ausgangskanal entspricht und nimmt Eingaben von allen Kanälen im Eingabetensor entgegen.**

Wir implementieren eine Kreuzkorrelationsfunktion
zur **Berechnung der Ausgabe mehrerer Kanäle** wie unten gezeigt.

In [ ]:
def corr2d_multi_in_out(X, K):
    # Iterate through the 0th dimension of `K`, and each time, perform
    # cross-correlation operations with input `X`. All of the results are
    # stacked together
    return tf.stack([corr2d_multi_in(X, k) for k in K], 0)

Wir konstruieren einen Faltungs-Kernel mit 3 Ausgangskanälen
durch Verkettung des Kernel-Tensors `K` mit `K+1`
(plus eins für jedes Element in `K`) und `K+2`.


In [ ]:
K = tf.stack((K, K + 1, K + 2), 0)
K.shape

Im Folgenden führen wir die Kreuzkorrelation auf den Eingabetensor `X` mit dem Kernel-Tensor `K` aus.
Die Ausgabe enthält nun 3 Kanäle. 

Das Ergebnis des ersten Kanals ist konsistent mit dem Ergebnis des vorherigen Eingangstensors `X`
und dem Multi-Input-Single-Output-Kernel.



In [ ]:
corr2d_multi_in_out(X, K)